In [7]:
import Pkg; Pkg.activate("/Users/boyanxu/Src/working/Pandemic-Sim/Agents.jl-examples/Project.toml")

  Activating environment at `~/Src/working/Pandemic-Sim/Agents.jl-examples/Project.toml`


In [8]:
using Agents

@agent SchellingAgent GridAgent{2} begin
    mood::Bool          # whether the agent is happy in its position. (true = happy)
    group::Int          # The group of the agent, determines mood as it interacts with neighbors    
end


using Random
function initialize(; numagents = 320, griddims = (20, 20), min_to_be_happy = 3, seed = 125)
    space = GridSpace(griddims; periodic=false)
    properties = Dict(:min_to_be_happy => min_to_be_happy)
    rng = Random.MersenneTwister(seed)
    
    model = ABM(
        SchellingAgent, space;
        properties, rng, scheduler = Schedulers.randomly
    )
    
    for n in 1:numagents
        agent = SchellingAgent(n, (1,1), false, n < numagents / 2 ? 1 : 2)
        add_agent_single!(agent, model)
    end
    return model
end

initialize (generic function with 1 method)

In [13]:
function agent_step!(agent, model)
    minhappy = model.min_to_be_happy
    count_neighbors_same_group = 0
    for neighbor in nearby_agents(agent, model)
        if agent.group == neighbor.group
            count_neighbors_same_group += 1
        end
    end
    
    if count_neighbors_same_group >= minhappy
        agent.mood = true
    else
        move_agent_single!(agent, model)
    end
    return
end

agent_step! (generic function with 1 method)

In [14]:
model = initialize()

AgentBasedModel with 320 agents of type SchellingAgent
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=false
 scheduler: randomly
 properties: min_to_be_happy

In [16]:
step!(model, agent_step!, 3)

In [17]:
adata = [:pos, :mood, :group]